# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
document = ET.parse( './data/mondial_database.xml' )

In [3]:
#1. 10 countries with the lowest infant mortality rates
infant_mortality = []
country = []
for child in document.getroot():
    try: 
        infant_mortality.append(child.find('infant_mortality').text)
        country.append(child.find('name').text)
    except AttributeError:  
        pass
im = pd.DataFrame(np.column_stack((infant_mortality, country)))
im = im.rename(columns={0:"rate", 1:"country"})
#print(type(im.rate[0])) #make sure the type is str or float before sorting
im.rate = im.rate.astype(float)
im.sort_values(by="rate").head(10)

,rate,country
36,1.81,Monaco
90,2.13,Japan
109,2.48,Bermuda
34,2.48,Norway
98,2.53,Singapore
35,2.60,Sweden
8,2.63,Czech Republic
72,2.73,Hong Kong
73,3.13,Macao
39,3.15,Iceland


In [4]:
#2. 10 cities with the largest population
population = []
city = []
for sub in document.getiterator('city'): 
    try:
        population.append(sub.findall('population')[-1].text) #population must try to be appended first so that 
        #if population does not exist it will pass before appending city name into the list. 
        #[-1]:get the latest year's population.
        city.append(sub.find('name').text)
    except AttributeError:  
        pass
    except IndexError: #if population is not present this error will raise since we're extracting [-1]
        pass
city_pop = pd.DataFrame(np.column_stack((population, city)))
city_pop = city_pop.rename(columns={0:"population", 1:"city"})
city_pop.population = city_pop.population.astype(int)
city_pop.sort_values(by="population", ascending=False).head(10)

,population,city
1251,22315474,Shanghai
707,13710512,Istanbul
1421,12442373,Mumbai
443,11979529,Moskva
1250,11716620,Beijing
2594,11152344,São Paulo
1252,11090314,Tianjin
974,11071424,Guangzhou
1467,11034555,Delhi
977,10358381,Shenzhen


In [5]:
#3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
ethnic_group = {}
for country in document.getiterator('country'): 
    try: 
        population = int(country.findall('population')[-1].text)
    except AttributeError:  
        pass
    except IndexError: 
        pass
    for ethnic in country.findall('ethnicgroup'):
        if ethnic.text in ethnic_group:
            ethnic_group[ethnic.text] += int(population * (float(ethnic.attrib['percentage']) / 100))
        else:
            ethnic_group[ethnic.text] = int(population * (float(ethnic.attrib['percentage']) / 100))
        
eg = pd.DataFrame.from_dict(ethnic_group, orient='index')
eg = eg.rename(columns={0:'population'})
eg.sort_values(by='population', ascending=False).head(10)

,population
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212
Malay,121993548


In [6]:
#combine the car_code of each country with its complete name
country_codes = {}
for country in document.getiterator('country'): 
    country_codes[country.attrib['car_code']] = country.find('name').text

In [7]:
#4. name and country of a) longest river
rivers = []
lengths = []
countries1 = []
for river in document.getiterator('river'):
    c = []
    try:
        lengths.append(river.find('length').text)
        rivers.append(river.find('name').text) # or river.attrib['id']
        country = river.attrib['country']
        for count in country.split(' '):
            count = country_codes[count]
            c.append(count)
        c = ', '.join(c)
        countries1.append(c)
    except AttributeError: 
        pass

river = pd.DataFrame(np.column_stack((rivers, countries1, lengths)))
river = river.rename(columns={0:"river", 1:"countries", 2:"length"})
river.length = river.length.astype(float)
river.sort_values(by="length", ascending=False).head(1)

,river,countries,length
174,Amazonas,"Colombia, Brazil, Peru",6448.0


In [8]:
#4. name and country of b) largest lake
lakes = []
areas = []
countries2 = []
for lake in document.getiterator('lake'):
    c = []
    try:
        areas.append(lake.find('area').text)
        lakes.append(lake.find('name').text) #lake.attrib['id']
        country = lake.attrib['country']
        for count in country.split(' '):
            count = country_codes[count]
            c.append(count)
        c = ', '.join(c)
        countries2.append(c)
    except AttributeError: 
        pass
lake = pd.DataFrame(np.column_stack((lakes, countries2, areas)))
lake = lake.rename(columns={0:"river", 1:"countries", 2:"area"})
lake.area = lake.area.astype(float)
lake.sort_values(by="area", ascending=False).head(1)

,river,countries,area
54,Caspian Sea,"Russia, Azerbaijan, Kazakhstan, Iran, Turkmeni...",386400.0


In [9]:
#4. name and country of c) airport at highest elevation
airports = {}
for airport in document.getiterator('airport'):
    try:
        elevation = float(airport.find('elevation').text)
        airport_name = airport.find('name').text
        country = airport.attrib['country']
        key = airport_name + ', ' + country_codes[country]
        airports[key] = elevation
    except AttributeError: 
        pass
    except TypeError:
        pass
a = pd.DataFrame.from_dict(airports, orient='index')
a = a.rename(columns={0:'elevation'})
a.sort_values(by="elevation", ascending=False).head(1)

,elevation
"El Alto Intl, Bolivia",4063.0
